# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/perplexity-ai/r1-1776',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/OmniParser-v2.0',
 '/stepfun-ai/Step-Audio-Chat',
 '/stepfun-ai/stepvideo-t2v',
 '/models',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces/microsoft/OmniParser-v2',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/m-ric/open_Deep-Research',
 '/spaces',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/facebook/natural_reasoning',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/SakanaAI/AI-CUDA-Engineer-Archive',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microso

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on mo

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nperplexity-ai/r1-1776\nUpdated\n3 days ago\n•\n7.03k\n•\n1.38k\ndeepseek-ai/DeepSeek-R1\nUpdated\n13 days ago\n•\n4.4M\n•\n9.9k\nmicrosoft/OmniParser-v2.0\nUpdated\n4 days ago\n•\n4.2k\n•\n834\nstepfun-ai/Step-Audio-Chat\nUpdated\n5 days ago\n•\n609\n•\n335\nstepfun-ai/stepvideo-t2v\nUpdated\n3 days ago\n•\n944\n•\n281\nBrowse 1M+ models\nSpaces\nRunning\n1.25k\n1.25k\nThe Ultra-Scale Playbook

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## About Us
Hugging Face is at the forefront of the AI revolution, committed to building an open, collaborative platform that empowers the machine learning (ML) community. We provide tools and resources that allow users to create, discover, and collaborate on models, datasets, and applications. With over **1 million models** and **250,000 datasets**, we are the definitive hub for AI innovation.

### Mission Statement:
**"The AI community building the future."**

---

## Our Offerings
### Key Features:
- **Models**: Collaborate with over 1M models including those from notable organizations like Amazon, Microsoft, and Google.
- **Datasets**: Access a rich collection of more than 250k datasets to fuel your AI projects.
- **Spaces**: Share and run your applications in an interactive environment.
- **Enterprise Solutions**: Tailored features for organizations with a strong focus on security, control, and dedicated support.

---

## Who We Serve
Hugging Face is trusted by **more than 50,000 organizations**, including top-tier players such as:
- **Ai2**
- **Meta**
- **Amazon Web Services**
- **Intel**
- **Grammarly**

Our platform provides the tools necessary for companies and researchers to leverage machine learning for diverse applications.

---

## Company Culture
We foster a **collaborative and innovative environment** that encourages continuous learning and sharing among team members. Hugging Face is built on the principles of **open source**, transparency, and community engagement. With a team of **210+** dedicated individuals, we thrive on pushing the boundaries of what's possible in AI and ML.

### Values:
- **Inclusivity & Diversity**: We believe that diverse perspectives foster innovation.
- **Community-Driven**: Contributions from users across the globe shape our offerings.
- **Learning & Growth**: We support our team's professional development and learning journeys.

---

## Careers at Hugging Face
Join us in our mission to democratize AI! We are always on the lookout for talented individuals who are passionate about ML and AI. Explore a variety of career opportunities ranging from engineering to community management.

- **Current Openings**: Visit [Hugging Face Careers](https://huggingface.co/jobs) to find positions suited for you.
- **Benefits**: We offer competitive salaries, flexible working hours, and a vibrant team culture.

---

## Connect with Us
Stay up to date with our latest developments and community activities:
- **Website**: [Hugging Face](https://huggingface.co)
- **GitHub**: [Hugging Face GitHub](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **Join our Discord**: Get involved and bring your ideas!

---

At Hugging Face, we are not just building tools; we are creating a community structured around innovation, collaboration, and shared success in AI. Discover the future with us!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

### The AI Community Building the Future

Hugging Face is at the forefront of the artificial intelligence revolution. Our platform serves as a collaborative space for the machine learning community, allowing users to create, discover, and contribute unparalleled models, datasets, and applications.

**Explore Our Offerings:**
- **Models:** Access over 1 million models across various domains.
- **Datasets:** Browse through 250k+ datasets to enhance your projects.
- **Spaces:** Run and showcase your innovative AI applications seamlessly.

---

## Our Commitment to Collaboration

At Hugging Face, we believe in the power of community. We provide a space for ML professionals, researchers, and enthusiasts to share their insights and collaborate on projects. Our open-source ecosystem empowers users with tools like Transformers, Diffusers, Tokenizers, and more, supporting diverse tasks in machine learning.

### Join our community, partner with top organizations:
Some of the notable organizations using Hugging Face’s platform include:
- **Microsoft**
- **Google**
- **Amazon Web Services**
- **Meta**
- **Intel**

With over 50,000 organizations leveraging our technology, the possibilities are endless.

---

## Cultivating a Unique Company Culture

### We Foster Innovation

Hugging Face not only encourages collaboration but thrives on innovation. Our work culture supports continual learning and experimentation. Employees are inspired to explore new ideas, making breakthroughs in AI possible.

### Values and Inclusivity

We value diversity and inclusivity, celebrating the unique perspectives that each member brings to the team. Our engagement in various community initiatives aims to promote learning and access in AI fields for everyone.

---

## Careers at Hugging Face

Hugging Face is constantly seeking talented individuals to join our team. As a growing company, we offer various career opportunities in machine learning, software development, product management, and more. 

### Why Work with Us?
- **Impact:** Your work will directly contribute to the AI landscape.
- **Growth:** We support and invest in your personal and professional development.
- **Collaboration:** Work alongside industry leaders and pioneers in the AI community.

Interested in joining us? Check out our [Careers page](#) for current job openings.

---

## Get Involved

Whether you’re a researcher, developer, business, or simply interested in AI, Hugging Face welcomes you to join our community. Explore applications, contribute to open-source projects, or engage in discussions on machine learning.

### Let’s Build the Future of AI Together!

For more information:
- Visit our website: [huggingface.co](https://huggingface.co)
- Connect on [Twitter](https://twitter.com/huggingface)
- Join the conversation on [Discord](https://discord.gg/huggingface)

Hugging Face – Together, we’re building the future of AI!

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face!
**The AI community building the future.** Hugging Face is the premier platform for collaboration in the machine learning community, providing tools and solutions for models, datasets, and applications. Join us in exploring and advancing the field of artificial intelligence.

---

## Company Overview
- **Founded**: Established with the vision to democratize AI and make it accessible to all.
- **Mission**: To foster a vibrant community that collaborates on building state-of-the-art machine learning models and applications.

---

## Our Offerings

### Models
Hugging Face hosts over **1 million models** across various tasks including text, image, video, and audio processing. Our community shares cutting-edge models for all to access and implement.

### Datasets
Explore and contribute to **250,000+ datasets**. Users can access a rich repository of data essential for training and developing AI models.

### Spaces
Hugging Face provides a collaboration platform to host and manage applications with ease. With **thousands of active applications,** you can experiment, build, and share your work.

### Enterprise Solutions
For organizations looking for robust support:
- Tailored enterprise plans starting from **$20/user/month**
- Advanced security features and dedicated support
- Over **50,000+ organizations** are already part of our community.

---

## Customers
Our diverse customer base includes renowned companies and organizations:
- **Microsoft**
- **Google**
- **Amazon Web Services**
- **Intel**
- **Grammarly**
- **Meta** (formerly Facebook)

---

## Company Culture
Hugging Face embraces a culture of **open-source collaboration** and **innovation**. We believe in the power of community and provide our team members with an environment that promotes:
- Continuous learning and experimentation
- Flexibility and creativity in problem-solving
- An open and inclusive atmosphere where every voice matters

---

## Career Opportunities
Join us on our mission! We are continuously looking for talent in various domains:
- **Machine Learning Engineers**
- **Data Scientists**
- **Software Developers**
- **Community Managers**

At Hugging Face, you will be part of a collaborative team that is passionate about AI and shaping the future of technology.

---

## Get Involved
Whether you’re a developer, a researcher, or a business leader, Hugging Face offers tools and resources to help you innovate.  
- **Join the Community**: Sign up on our platform to start collaborating.  
- **Explore**: Check our documentation and community forum to learn more.  
- **Follow Us**: Stay updated on our activities through GitHub, Twitter, LinkedIn, and Discord.

---

### Contact Us
- **Website**: [huggingface.co](https://huggingface.co)
- **Social Media**: Connect with us on social platforms to get the latest updates.

Join Hugging Face and be a part of revolutionizing AI today!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>